In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
file_name = "df_count_4.csv"   

In [3]:
 
df = pd.read_csv(file_name , parse_dates=["date"])


C:\Users\oussa\AppData\Local\Temp\ipykernel_22948\1571972478.py:1: DtypeWarning: Columns (0,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name , parse_dates=["date"])


In [4]:
df.shape

(9170982, 10)

In [5]:
# Read the country reference file
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

In [6]:
# Define columns to ignore when checking duplicates
cols_ignore = ["RECPTCL_FID", "duration_to_next_step", "total_duration", "id"]
cols_check  = [c for c in df.columns if c not in cols_ignore]

# 1. Identify all duplicate rows based on cols_check
dup_mask = df.duplicated(subset=cols_check, keep=False)

# 2. Number of such duplicate rows
num_duplicates = dup_mask.sum()
print(f"Number of duplicate rows (ignoring specified columns): {num_duplicates}")




Number of duplicate rows (ignoring specified columns): 2465487


In [7]:
df = df.drop_duplicates(subset=cols_check, keep="first").reset_index(drop=True)

In [15]:
df.shape

(7938238, 7)

In [8]:
import pandas as pd

ID_COL    = "MAILITM_FID"
EVENT_COL = "EVENT_TYPE_NM"

# 1) pick the first row per parcel (rows are already in correct order)
first_events = (
    df.groupby(ID_COL, sort=False)[EVENT_COL]
      .first()                 # a Series: index = parcel ID, value = first event
)

# 2) count how many parcels have each first event
event_counts = (
    first_events.value_counts()   # Series: index = event, value = count
               .sort_values(ascending=False)
)

# 3) print the result
print("First-event frequencies:")
for event, count in event_counts.items():
    print(f"{event:40} {count:,}")


First-event frequencies:
Insérer envoi dans sac (Srt)             702,223
Réception d'envoi du client (Srt)        28,391
Expédition d'envoi à l'étranger (EDI-reçu) 51


In [9]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

RECPTCL_FID                   779889
établissement_postal           45580
next_établissement_postal    5188569
duration_to_next_step        2637256
dtype: int64


In [10]:
df = df.drop(columns=["RECPTCL_FID", "duration_to_next_step", "total_duration"])

In [11]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

établissement_postal           45580
next_établissement_postal    5188569
dtype: int64


In [12]:

# Assuming 'df' is your DataFrame!

# Identify the rows with missing values
missing_rows = df[df.isnull().any(axis=1)]

# Print the total number of rows that have missing values
print("Total rows with missing values:", missing_rows.shape[0])




Total rows with missing values: 5208531


In [13]:
# Analyze the missing patterns across rows:
# Create a binary pattern for each row where 1 indicates a missing value and 0 indicates non-missing.
missing_pattern = missing_rows.isnull().astype(int)
# Convert each pattern to a tuple to use as a key for groupby
missing_pattern_tuples = missing_pattern.apply(tuple, axis=1)
# Group by the missing pattern and count the number of rows for each pattern
pattern_summary = missing_rows.groupby(missing_pattern_tuples).size().sort_values(ascending=False)

print("\nSummary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):")
print(pattern_summary)




Summary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):
(0, 0, 0, 0, 0, 1, 0)    5162951
(0, 0, 0, 1, 0, 1, 0)      25618
(0, 0, 0, 1, 0, 0, 0)      19962
dtype: int64


In [14]:
distinct_values_count = df.nunique()

distinct_values_count = distinct_values_count.sort_values(ascending=True)

print(distinct_values_count)

EVENT_TYPE_NM                     24
EVENT_TYPE_CD                     24
id                                57
next_établissement_postal       3378
établissement_postal            3539
MAILITM_FID                   730665
date                         6160639
dtype: int64


In [16]:

# Count the occurrences of each protocol
event_type = df['EVENT_TYPE_NM'].value_counts()

print(event_type)


EVENT_TYPE_NM
Recevoir envoi au bureau de livraison (Ent)                    1536891
Expédier envoi à adresse nationale (Ent)                       1048919
Expédier envoi à adresse nationale (Srt)                        952032
Vaine tentative de livraison d'envoi (Ent)                      894172
Insérer envoi dans sac (Srt)                                    758703
Recevoir envoi au bureau d'échange (Ent)                        744151
Expédition d'envoi à l'étranger (EDI-reçu)                      733535
Livraison d'envoi (Ent)                                         690459
Transmettre envoi à l'agent de livraison (Ent)                  413004
Enregistrer détails d'envoi au bureau d'échange (Srt)           106371
Réception d'envoi du client (Srt)                                29788
Expédier envoi à la douane (Ent)                                 15041
Renvoyer envoi de la douane (Ent)                                 7054
Recevoir envoi au bureau d'échange (Srt)                       

In [17]:

# Count the occurrences of each protocol
proto_counts = df['établissement_postal'].value_counts()

print(proto_counts)


établissement_postal
SINGAPOUR                    1410261
ALGER GARE                    895155
ALGER COLIS POSTAUX           829610
CONSTANTINE COLIS POSTAUX     281558
ANNABA EL MARSA               172956
                              ...   
Agence EMS SILA                    1
BOUDJELIKH                         1
TAGHOUZI                           1
RAS-EL-AIN                         1
GARIDI 1B                          1
Name: count, Length: 3539, dtype: int64


In [18]:
# Count the occurrences of each protocol
proto_counts = df['next_établissement_postal'].value_counts()

print(proto_counts)



next_établissement_postal
ALGÉRIE                      733535
ALGER GARE                   254897
CONSTANTINE COLIS POSTAUX    108377
ORAN COLIS POSTAUX           104986
ANNABA EL MARSA               83783
                              ...  
EL-ATHMANIA                       1
DJEMAA                            1
ZEMOURA                           1
NADOR-M NAOUER                    1
OULED FAYET-02                    1
Name: count, Length: 3378, dtype: int64


In [19]:

# Count the occurrences of each protocol
proto_counts = df['id'].value_counts()

print(proto_counts)

s = proto_counts


id
SG    7537421
TR     124549
SE      95530
US      92173
TN      53692
UA       6946
RO       5240
RU       4792
FR       4729
TH       3896
NL       2008
BG       1203
BA        654
PS        612
PK        522
DE        500
AZ        500
VN        350
LY        344
BY        322
AL        253
GB        251
MY        230
MK        193
TG        133
SN        117
ES        107
LU        104
LT         99
CN         86
IQ         83
CA         64
MR         60
LV         55
CZ         49
IT         40
BH         35
MA         27
JE         24
MN         22
ML         20
IN         20
PT         18
KZ         17
CH         15
CI         14
IE         13
BF         12
MX         12
SA         12
JP         12
KG         11
HU         10
HR         10
DJ          9
VA          9
OM          9
Name: count, dtype: int64


In [20]:
# Map the Series index using the country_dict
s.index = s.index.map(lambda code: country_dict.get(code, code))  # Keep unmapped codes as-is
print(s)

id
SINGAPOUR                                   7537421
TURQUIE                                      124549
SUÈDE                                         95530
ÉTATS-UNIS                                    92173
TUNISIE                                       53692
UKRAINE                                        6946
ROUMANIE                                       5240
RUSSIE, FÉDÉRATION DE                          4792
FRANCE                                         4729
THAÏLANDE                                      3896
PAYS-BAS                                       2008
BULGARIE                                       1203
BOSNIE-HERZÉGOVINE                              654
PALESTINIEN OCCUPÉ, TERRITOIRE                  612
PAKISTAN                                        522
ALLEMAGNE                                       500
AZERBAÏDJAN                                     500
VIET NAM                                        350
LIBYENNE, JAMAHIRIYA ARABE                      344
BÉLARUS  

In [21]:
print("Min date:", df["date"].min())
print("Max date:", df["date"].max())

Min date: 2020-12-14 12:20:32.043
Max date: 2025-05-21 15:54:58.333


In [22]:
df["date"] = df["date"].str.replace(r'\.\d+$', '', regex=True)
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)

C:\Users\oussa\AppData\Local\Temp\ipykernel_22948\579475394.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)


In [23]:
df = df.sort_values(by=["MAILITM_FID", "date"])

# Calculate time to next step
df["duration_to_next_step"] = df.groupby("MAILITM_FID")["date"].shift(-1) - df["date"]

# Calculate total duration per ID
first_date = df.groupby("MAILITM_FID")["date"].transform("first")
last_date = df.groupby("MAILITM_FID")["date"].transform("last")
df["total_duration"] = last_date - first_date




In [24]:
# Preview the updated dataset
df[["MAILITM_FID", "date", "EVENT_TYPE_NM", "duration_to_next_step", "total_duration"]].head(10)

,MAILITM_FID,date,EVENT_TYPE_NM,duration_to_next_step,total_duration
0,CA000020800LY,2022-02-03 10:19:00,Insérer envoi dans sac (Srt),6 days 22:15:00,24 days 23:11:23
1,CA000020800LY,2022-02-10 08:34:00,Insérer envoi dans sac (Srt),2 days 05:41:00,24 days 23:11:23
780852,CA000020800LY,2022-02-12 14:15:00,Expédition d'envoi à l'étranger (EDI-reçu),9 days 19:35:25,24 days 23:11:23
3499208,CA000020800LY,2022-02-22 09:50:25,Recevoir envoi au bureau d'échange (Ent),0 days 00:01:38,24 days 23:11:23
3499209,CA000020800LY,2022-02-22 09:52:03,Expédier envoi à adresse nationale (Ent),0 days 00:42:49,24 days 23:11:23
3499210,CA000020800LY,2022-02-22 10:34:52,Expédier envoi à adresse nationale (Ent),1 days 23:56:45,24 days 23:11:23
3499211,CA000020800LY,2022-02-24 10:31:37,Expédier envoi à adresse nationale (Srt),1 days 23:49:42,24 days 23:11:23
3499212,CA000020800LY,2022-02-26 10:21:19,Recevoir envoi au bureau de livraison (Ent),0 days 00:02:44,24 days 23:11:23
3499213,CA000020800LY,2022-02-26 10:24:03,Expédier envoi à adresse nationale (Srt),0 days 23:16:12,24 days 23:11:23
3499214,CA000020800LY,2022-02-27 09:40:15,Recevoir envoi au bureau de livraison (Ent),0 days 00:00:35,24 days 23:11:23


In [25]:
# Filter rows where total_duration is NOT zero
zero_duration_df = df[df["total_duration"] <= pd.Timedelta(days = 1)]

# Display the first few rows as a check
zero_duration_df.shape


(0, 9)

In [38]:

# Count the occurrences of each protocol
countries = zero_duration_df['id'].value_counts()

print(countries)


Series([], Name: count, dtype: int64)


In [39]:
# Map the Series index using the country_dict
countries.index = countries.index.map(lambda code: country_dict.get(code, code))  
print(countries)

Series([], Name: count, dtype: int64)


In [40]:
# Step 1: Count how many times each ID appears
id_counts = zero_duration_df["MAILITM_FID"].value_counts()

# Step 2: Count how many IDs have a specific count
count_of_counts = id_counts.value_counts().sort_index()

print(count_of_counts)


Series([], Name: count, dtype: int64)


In [41]:
ids_with_0 = id_counts[id_counts == 2].index
df_with_0 = df[df["MAILITM_FID"].isin(ids_with_0)]
df_with_0.head()

,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal,id,duration_to_next_step,total_duration


In [42]:

# Count the occurrences of each protocol
event_type = df_with_0['EVENT_TYPE_NM'].value_counts()

print(event_type)



Series([], Name: count, dtype: int64)


In [26]:
# Drop rows where total_duration is zero
df = df[df["total_duration"] != pd.Timedelta(0)]

# Check the shape after dropping
print(df.shape)


(7938238, 9)


In [27]:
df.to_csv(file_name, index=False, encoding='utf-8-sig')